<a href="https://colab.research.google.com/github/makhmudov-khondamir/Machine-Learning-Projects/blob/main/13.Multi-Category%20Image%20Classification%20with%20Custom%20Dataset%20Using%20fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Title: Multi-Category Image Classification with Custom Dataset Using fastai**

### Description:
This project demonstrates how to build and train an image classification model for multiple categories using the fastai library. The example uses a custom dataset from the Open Images Dataset (OIDv4) to classify images into different transport categories.

### Dataset:
The dataset used in this project is a subset of the Open Images Dataset (OIDv4), which contains images categorized into various classes. In this example, we use categories related to different types of transport.

In [ ]:
import torch
import fastai

# Dataset. We use Google's Open Images Dataset v4 to download the dataset.  This dataset consists of 600 classes and contains 1.7 million images.
# We use OIDv4_Toolkit to download these images.  (https://github.com/EscVM/OIDv4_ToolKi)
! git clone https://github.com/EscVM/OIDv4_ToolKit.git

# This installs the necessary Python packages (pandas,requests,tqdm,argparse,Pillow,numpy,sh..etc) listed in the requirements.txt file from the OIDv4 ToolKit.
!cd OIDv4_ToolKit && pip install -r requirements.txt

For this project, we will build a model that can classify three types of vehicles (cars, airplanes, water transports (ships, boats)).  
That's why we download 200 images for each class using OlDv4 Toolkit.

In [ ]:
!cd OIDv4_ToolKit && python3 main.py downloader --Dataset /content --classes Car Airplane Boat --type_csv train --limit 200

## **Building a Model**

In [ ]:
from fastai.vision.all import *
from ipywidgets import widgets

path=Path('train')  # train is the directory containing those 200 training images for our project. It is located in Files
path.ls() # *it shows these 3 files included

fls=get_image_files(path)
fls # *it shows these all images included

failed=verify_images(path) # *Checks for corrupted image files in the specified directory.
failed

transports=DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(224)
)

# Dataloader
dls=transports.dataloaders(path)

#check
dls.train.show_batch(max_n=8, nrows=2)

learn=cnn_learner(dls,resnet34, metrics=accuracy)
learn.fine_tune(4)

interp=ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

interp.plot_top_losses(5, nrows=1)

In [ ]:
upload=widgets.FileUpload()
upload

In [ ]:
img=PILImage.create(upload.data[-1])
pred, pred_id, prob=learn.predict(img)
print(f'Prediction: {pred}')
print(f'Probability: {prob[pred_id]}')
img

### Code Explanation:

```python
import torch
import fastai

!git clone https://github.com/EscVM/OIDv4_ToolKit.git

!cd OIDv4_ToolKit && pip install -r requirements.txt
```

#### What Happens in This Step:

1. **Cloning the OIDv4 ToolKit**:
   - `!git clone https://github.com/EscVM/OIDv4_ToolKit.git`: This command clones the OIDv4 ToolKit repository, which provides tools to download images from the Open Images Dataset.

2. **Installing Requirements**:
   - `!cd OIDv4_ToolKit && pip install -r requirements.txt`: This installs the necessary Python packages listed in the `requirements.txt` file from the OIDv4 ToolKit.

#### Real-Life Example:
Think of this as downloading a tool from the internet to help you gather and organize images from a large online database.

```python
from fastai.vision.all import *
from ipywidgets import widgets

#path
path=Path('train')
path.ls()
fls=get_image_files(path)
fls
```

#### What Happens in This Step:

1. **Setting the Path**:
   - `path = Path('train')`: Specifies the directory where the training images are stored.
   - `path.ls()`: Lists all the files and directories in the 'train' directory.

2. **Getting Image Files**:
   - `fls = get_image_files(path)`: Retrieves all image file paths in the specified directory.

#### Real-Life Example:
Imagine you have a folder named 'train' that contains images of different types of transport. This step identifies all image files in that folder.

```python
failed = verify_images(path)
failed
```

#### What Happens in This Step:

1. **Verifying Images**:
   - `failed = verify_images(path)`: Checks for corrupted image files in the specified directory.
   - `failed`: Lists any files that failed the verification check.

#### Real-Life Example:
Consider this step as a quality check to ensure all images in your folder are readable and not corrupted.

```python
transports = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(224)
)
```

#### What Happens in This Step:

1. **Defining the DataBlock**:
   - `blocks=(ImageBlock, CategoryBlock)`: Specifies the types of data: images and their categories.
   - `get_items=get_image_files`: Function to get image files.
   - `splitter=RandomSplitter(valid_pct=0.2, seed=42)`: Randomly splits the data into training and validation sets, with 20% for validation.
   - `get_y=parent_label`: Uses the parent folder name as the label for each image.
   - `item_tfms=Resize(224)`: Resizes all images to 224x224 pixels.



```python
# Dataloader
dls = transports.dataloaders(path)

#check
dls.train.show_batch(max_n=8, nrows=2)
```

#### What Happens in This Step:

1. **Creating DataLoaders**:
   - `dls = transports.dataloaders(path)`: Creates DataLoaders for the training and validation datasets.

2. **Displaying a Batch of Images**:
   - `dls.train.show_batch(max_n=8, nrows=2)`: Displays a batch of 8 training images in 2 rows.

#### Real-Life Example:
This step loads the images into memory for training and displays a sample batch to visually confirm the data and labels.

```python
learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(4)
```

#### What Happens in This Step:

1. **Creating and Training the Model**:
   - `cnn_learner(dls, resnet34, metrics=accuracy)`: Creates a Convolutional Neural Network (CNN) learner using the ResNet-34 architecture.
   - `learn.fine_tune(4)`: Fine-tunes the model for 4 epochs.

#### Real-Life Example:
Think of this step as teaching a pre-trained neural network to recognize different types of transport using your dataset, refining its knowledge over 4 iterations.

```python
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

interp.plot_top_losses(5, nrows=1)
```

#### What Happens in This Step:

1. **Interpreting the Model**:
   - `ClassificationInterpretation.from_learner(learn)`: Creates an interpretation object for the trained model.
   - `interp.plot_confusion_matrix()`: Plots the confusion matrix to visualize the model's performance.
   - `interp.plot_top_losses(5, nrows=1)`: Displays the top 5 images where the model's predictions were most incorrect.

#### Real-Life Example:
Consider this step as analyzing the model's performance, identifying areas where it performs well and where it makes mistakes.

```python
upload = widgets.FileUpload()
upload

img = PILImage.create(upload.data[-1])
pred, pred_id, prob = learn.predict(img)
print(f'Prediction: {pred}')
print(f'Probability: {prob[pred_id]}')
img
```

#### What Happens in This Step:

1. **Uploading and Predicting**:
   - `upload = widgets.FileUpload()`: Creates a file upload widget.
   - `img = PILImage.create(upload.data[-1])`: Creates an image object from the uploaded file.
   - `pred, pred_id, prob = learn.predict(img)`: Uses the trained model to predict the category of the uploaded image.
   - `print(f'Prediction: {pred}')`: Prints the predicted category.
   - `print(f'Probability: {prob[pred_id]}')`: Prints the probability of the predicted category.


This step allows you to upload a new image, have the model predict its category, and see the prediction and its probability.